In [1]:
import os
import sys

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config("spark.sql.warehouse.dir", "C:/Users/G312")\
    .getOrCreate()

In [2]:
marks=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]
_marksRdd=spark.sparkContext.parallelize(marks)

In [4]:
#문제 1

In [16]:
marksByName = _marksRdd\
        .map(lambda x: x.split(','))\
        .map(lambda x: (x[0], float(x[2])))\
        .reduceByKey(lambda x,y : x+y)\
        .collect()

for res in marksByName:
    print res[0], res[1]

'임하나' 170.0
'김하나' 180.0
'김갑돌' 180.8


In [6]:
#문제 2

In [17]:
marksBySubject = _marksRdd\
        .map(lambda x: x.split(','))\
        .map(lambda x: (x[1], float(x[2])))\
        .reduceByKey(lambda x,y : x+y)\
        .collect()

for res in marksBySubject:
    print res[0], res[1]

'English' 252.3
'Math' 278.5


In [10]:
#문제 3

In [15]:
_marksbyname2=_marksRdd\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))

marks_collect=_marksbyname2.combineByKey(lambda value: (value,1),
                             lambda x,value: (x[0]+value, x[1]+1),
                             lambda x,y: (x[0]+y[0], x[1]+y[1]))

for i in marks_collect.collect():
    for result in i:
        print result,
    print

'임하나' (170.0, 2)
'김하나' (180.0, 2)
'김갑돌' (180.8, 2)


In [12]:
#문제 4

In [14]:
averageByKey = marks_collect\
    .map(lambda (key,(sum,count)) : (key, float(sum)/count))\
    .collect()

for i in averageByKey:
    for j in i:
        print j,
    print

'임하나' 85.0
'김하나' 90.0
'김갑돌' 90.4
